In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle  
import os
import pandas as pd
import os
from pycaret.classification import load_model, predict_model
import datetime

In [ ]:
#Loading different datasets

#df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx'',sheet_name='can_tintorer', engine='openpyxl')
#df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx'',sheet_name='encinsasola', engine='openpyxl') 
df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx',sheet_name='aliste', engine='openpyxl')
df.info()


In [ ]:
#Loading models
full_model = load_model(model_path)
destilled_model = load_model(model_path)
#specific
VdH = load_model(model_path)
PQ = load_model(model_path)
French = load_model(model_path)

### Prediction_function

In [ ]:

def predict_from_excel(model_path, excel_path, exclude_columns=None, output_column='predictions'):
    """
    Carga un archivo Excel, añade una columna 'rand' con valores 0,
    hace predicciones usando un modelo pre-entrenado de PyCaret,
    y guarda los resultados en un nuevo archivo Excel.

    :param model_path: Ruta al archivo del modelo 
    :param excel_path: Ruta al archivo con los datos de entrada
    :param exclude_columns: Lista de columnas a excluir como entrada (opcional)
    :param output_column: Nombre de la columna para las predicciones (por defecto: 'predictions')
    :return: Ruta al archivo de salida con las predicciones
    """
    # Loading  the model usando PyCaret
    modelo = load_model(model_path)

    # Cargar los datos
    df = pd.read_excel(excel_path)

    # Añadir la columna 'rand' con valores 0
    #df['rand'] = 0

    # Excluir columnas si se especifican
    if exclude_columns:
        df = df.drop(columns=[col for col in exclude_columns], errors='ignore')

    # Hacer predicciones usando PyCaret
    predictions = predict_model(model, data=df,raw_score=True)
    
    # El resultado de predict_model ya incluye las predicciones, 
    # generalmente en una columna llamada 'Label' o similar
    # Renombrar la columna de predicciones si es necesario
    if 'Label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'Label': output_column})
    elif 'prediction_label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'prediction_label': output_column})
    else:
        # Si la columna de predicciones tiene otro nombre, ajusta esto según sea necesario
        pass

    # Eliminar la columna 'rand' del resultado final si no se necesita
    if 'rand' in predicciones.columns:
        predicciones = predicciones.drop(columns=['rand'])

    # Crear nombre para el archivo de salida
    base_name = os.path.splitext(excel_path)[0]
    current_date = datetime.datetime.now().strftime("%Y%m%d")
    output_path = f"{base_name}_{current_date}_predictions.xlsx"
   #output_path = f"{base_name}_con_predicciones.xlsx"

    # Guardar resultados
    predictions.to_excel(output_path, index=False)

    return output_path

In [ ]:

# Ejemplo de uso:
model_path = '20240827_SinCaDestilledVortex2SMOTECalibrated'
excel_path = "./DATA/VARISCITA ROMANA100.xlsx"
exclude_columns = []  # Columnas que no son para el modelo

try:
    output_file = predict_from_excel(model_path, excel_path, exclude_columns)
    print(f"Predictions saved in: {output_file}")
except Exception as e:
    print(f"Error processing the file: {str(e)}")